# Dublin Bike CA 1

## 2 Data Preparation Notebook


Ronan Downes | [Github](https://github.com/ronandownes/dublinbikes-CA1) | November 2022 

Prerequisite Notebook: **1 Data Loading Notebook**

Follow on Notebook: **3 Data Visualisation**
***

In [117]:
#import necessary libraries and files 
import pandas as pd
import numpy as np
import warnings
import time
import datetime as dt
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import folium
import sklearn
import seaborn as sns

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans


%matplotlib inline

### Convert CSVs to pandas dfs and clean up

In [ ]:
# q1_df=  pd.read_csv('data/21_1.csv')
# q2_df = pd.read_csv('data/21_2.csv')
# q3_df=  pd.read_csv('data/21_3.csv')
# q4_df = pd.read_csv('data/21_4.csv')
df = pd.read_csv('data/bikes.csv')
wdf=pd.read_csv('data/weather.csv')

In [101]:
df.nunique (axis=0, dropna=True)

STATION ID                   111
TIME                      103038
LAST UPDATED             5039168
NAME                         111
BIKE STANDS                   18
AVAILABLE BIKE STANDS         41
AVAILABLE BIKES               41
STATUS                         2
ADDRESS                      111
LATITUDE                     111
LONGITUDE                    111
dtype: int64

In [78]:
df= df[df['STATUS'] == 'Open']    #Effectively removing data from closed stations

False

In [80]:
df.nunique (axis=0, dropna=True) #check that only one "STATUS" remains

STATION ID                   111
TIME                      103038
LAST UPDATED             5038920
NAME                         111
BIKE STANDS                   18
AVAILABLE BIKE STANDS         41
AVAILABLE BIKES               41
STATUS                         1
ADDRESS                      111
LATITUDE                     111
LONGITUDE                    111
dtype: int64

In [102]:
# Remove column name 'TIME'
df=df.drop(['TIME'], axis=1)
df=df.drop(['STATUS'], axis=1)   # 774.8+ MB

In [104]:
#remove rows where no bike has been taken or returned since previous readings
df.drop_duplicates(keep= 'first',inplace=True)

In [105]:
df.info() #memory usage: 474.7+ MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6222487 entries, 0 to 11283233
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STATION ID             int64  
 1   LAST UPDATED           object 
 2   NAME                   object 
 3   BIKE STANDS            int64  
 4   AVAILABLE BIKE STANDS  int64  
 5   AVAILABLE BIKES        int64  
 6   ADDRESS                object 
 7   LATITUDE               float64
 8   LONGITUDE              float64
dtypes: float64(2), int64(4), object(3)
memory usage: 474.7+ MB


###  Filter to dates and removes duplicate rows
Usage of the DataFrame.loc[] Method to Filter Data to interval of interest and drop the "TIME" feature because it is reduntant.

The aim is to plan rebalancing and growth based on ML models so COVID-19 lockdown and xmas Holidays are ommitted.
Memory usage is 685 MB after Date filter and before merging weather data.

In [111]:
start_date = '2021-04-12'    #Lockdown restrictions lifted
end_date = '2021-12-11'      # Traditiona date for beginning of xmas holidays 
after_start_date = df['LAST UPDATED'] >= start_date
before_end_date = df['LAST UPDATED'] <= end_date
between_two_dates = after_start_date & before_end_date
# Using pandas.DataFrame.loc to Filter Rows by Dates
df = df.loc[between_two_dates]
df.drop_duplicates(keep= 'first',inplace=True)
df.info()   #memory usage: 319.6+ MB
df.shape # shape is (4188876, 9)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4188876 entries, 3109999 to 10611332
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STATION ID             int64  
 1   LAST UPDATED           object 
 2   NAME                   object 
 3   BIKE STANDS            int64  
 4   AVAILABLE BIKE STANDS  int64  
 5   AVAILABLE BIKES        int64  
 6   ADDRESS                object 
 7   LATITUDE               float64
 8   LONGITUDE              float64
dtypes: float64(2), int64(4), object(3)
memory usage: 319.6+ MB


(4188876, 9)

### Columns renamed using a  capitalized snake  style for readability

In [112]:
# remove spaces in columns name
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.lower()
df.columns = df.columns.str.capitalize()


In [113]:
df.head()

,Station_id,Last_updated,Name,Bike_stands,Available_bike_stands,Available_bikes,Address,Latitude,Longitude
3109999,2,2021-04-12 00:00:15,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110001,2,2021-04-12 00:10:21,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110003,2,2021-04-12 00:20:28,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110005,2,2021-04-12 00:30:34,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110007,2,2021-04-12 00:40:40,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814


In [ ]:
# sdf.rename(columns = {'last updated':'updated','available bike stands':'free_stands','available bikes':'bikes'}, inplace = True)
# sdf.head()

In [114]:
#Add individual date and time columns
df['Date_time'] = [dt.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df["Last_updated"]]
df['Last_updated'] = [dt.datetime.time(d) for d in df['Date_time']] 
df['Date'] = [dt.datetime.date(d) for d in df['Date_time']] 
df['Merge_date'] = df['Date_time'].dt.round('H')

In [98]:
df.head()

,Station_id,Last_updated,Name,Bike_stands,Available_bike_stands,Available_bikes,Status,Address,Latitude,Longitude,DATETIME,DATE,date_for_merge
3109999,2,00:00:15,BLESSINGTON STREET,20,8,12,Open,Blessington Street,53.35677,-6.26814,2021-04-12 00:00:15,2021-04-12,2021-04-12 00:00:00
3110001,2,00:10:21,BLESSINGTON STREET,20,8,12,Open,Blessington Street,53.35677,-6.26814,2021-04-12 00:10:21,2021-04-12,2021-04-12 00:00:00
3110003,2,00:20:28,BLESSINGTON STREET,20,8,12,Open,Blessington Street,53.35677,-6.26814,2021-04-12 00:20:28,2021-04-12,2021-04-12 00:00:00
3110005,2,00:30:34,BLESSINGTON STREET,20,8,12,Open,Blessington Street,53.35677,-6.26814,2021-04-12 00:30:34,2021-04-12,2021-04-12 01:00:00
3110007,2,00:40:40,BLESSINGTON STREET,20,8,12,Open,Blessington Street,53.35677,-6.26814,2021-04-12 00:40:40,2021-04-12,2021-04-12 01:00:00


In [ ]:
### Prepare Weather Data

weather = pd.read_csv('hly175.csv')
weather['date'] = [dt.datetime.strptime(d, "%m/%d/%Y %H:%M") for d in weather['date'] ]
weather['date_for_merge'] = weather['date'].dt.round('H')
weather = weather[(weather['date'] >= '2019-04-01') & (weather['date'] < '2020-04-02')]
weather = weather[['date_for_merge', 'rain', 'temp', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl']]
weather['rain'] = weather['rain'].astype(float)
weather['temp'] = weather['temp'].astype(float)

#add binary variables to note wet/dry weather and hot or not days (temperature)
weather['dry'] = np.where(weather['rain'] > 0.0, 1, 0)
weather['warm'] = np.where(weather['temp'] > 18.0, 1, 0)
weather.sample(5)


In [ ]:
#merge weather with bikes data
merged_data = pd.merge(data, weather, on = 'date_for_merge', how = 'left')

# identify bike arrivals and bike departures
merged_data['BIKE_ARR_DEP'] = merged_data.groupby('STATION ID')['AVAILABLE BIKE STANDS'].diff(-1)
merged_data['BIKE_ARR'] = np.where(merged_data['BIKE_ARR_DEP'] > 0, merged_data['BIKE_ARR_DEP'], 0)
merged_data['BIKE_DEP'] = np.where(merged_data['BIKE_ARR_DEP'] < 0, merged_data['BIKE_ARR_DEP'], 0)
merged_data['ACTIVITY_TYPE'] = np.where(abs(merged_data['BIKE_ARR_DEP']) >= 10, "REBALANCING", "RENTAL")
merged_data['IMBALANCED'] = np.where(merged_data['OCCUPANCY_PCT'] < .1, 1, 
                                   np.where(merged_data['OCCUPANCY_PCT'] > .9, 1,0 ))

# Identify days with rebalancing
merged_data['REBALANCING'] = np.where(merged_data['ACTIVITY_TYPE'] == 'REBALANCING', 1,0)
merged_data['JOIN_ON'] = merged_data['STATION ID'].apply(str)  + (merged_data['DATE']).apply(str) 
join_table= merged_data.groupby(['JOIN_ON'])['REBALANCING'].sum()
merged_data = merged_data.drop(['REBALANCING'], axis = 1)
join_table = join_table.to_frame()
join_table =join_table.reset_index()
merged_data = pd.merge(merged_data, join_table, on = 'JOIN_ON', how = 'left')
merged_data = merged_data.drop(['JOIN_ON'], axis = 1)

merged_data.sample(5)

### Time Clusters

In [ ]:
#group data into clusters
clustering_df = merged_data[['STATION ID', 'NAME', 'LATITUDE', 'LONGITUDE', 'DAY_TYPE', 'TIME_TYPE', 'OCCUPANCY_PCT','CLUSTER_GROUP']]
clustering_df = clustering_df.groupby(['STATION ID', 'NAME', 'LATITUDE', 'LONGITUDE', 'CLUSTER_GROUP'],as_index=False)['OCCUPANCY_PCT'].mean()
clustering_df  = clustering_df.set_index('STATION ID')

#pivot dataframe for clustering
clustering_df = clustering_df.pivot_table(index= ['NAME', 'STATION ID','LATITUDE', 'LONGITUDE'] , columns=['CLUSTER_GROUP'], values='OCCUPANCY_PCT')
clustering_df  = clustering_df.reset_index()
clustering_df  = clustering_df .set_index('NAME')
clustering_df = clustering_df.dropna()

clustering_df.sample(5)

### K-Means Clustering Analysis

In [ ]:
distortions = []
K = range(1,10)
X = np.array(clustering_df.drop(['STATION ID', 'LATITUDE', 'LONGITUDE'], 1).astype(float))
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    distortions.append(kmeanModel.inertia_)
    
plt.figure(figsize=(10,7))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
#clustering algo
X = np.array(clustering_df.drop(['STATION ID', 'LATITUDE', 'LONGITUDE'], 1).astype(float))
KM = KMeans(n_clusters=5) 
KM.fit(X)
clusters = KM.predict(X)

locations = clustering_df
locations['Cluster'] = clusters
locations = locations.reset_index()
locations.head(5)

In [ ]:
colordict = {0: 'blue', 1: 'red', 2: 'orange', 3: 'green', 4: 'purple'}
dublin_map = folium.Map([53.345, -6.2650], zoom_start=13.5)
for LATITUDE, LONGITUDE, Cluster in zip(locations['LATITUDE'],locations['LONGITUDE'], locations['Cluster']):
    folium.CircleMarker(
        [LATITUDE, LONGITUDE],
        color = 'b',
        radius = 8,
        fill_color=colordict[Cluster],
        fill=True,
        fill_opacity=0.9
        ).add_to(dublin_map)
dublin_map
    

In [ ]:
#merge clusters back into main dataset

merged_with_clusters = merged_data
cluster_output = locations[['STATION ID', 'Cluster']]
cluster_output.drop_duplicates(keep = 'first', inplace = True)
del merged_data
merged_with_clusters = pd.merge (merged_with_clusters, cluster_output, on = 'STATION ID', how = 'left')
merged_with_clusters['BIKE_ARR_DEP_ABS'] = abs(merged_with_clusters['BIKE_ARR_DEP'])
merged_with_clusters.sample(5)

In [ ]:
linechart_data = merged_with_clusters[['DAY_TYPE', 'Cluster', 'HOUR', 'OCCUPANCY_PCT']]
linechart_data['Cluster'] = np.where(linechart_data['Cluster'] == 0, 'City Centre',
                                    np.where(linechart_data['Cluster'] == 1, 'Grangegorman',
                                            np.where(linechart_data['Cluster'] == 2, 'Transport Hubs',
                                                    np.where(linechart_data['Cluster'] == 3, 'Docklands & South City', 'Outer Suburbs'))))

#Weekday
linechart_data_weekday = linechart_data[linechart_data['DAY_TYPE'] == 'Weekday']
linechart_data_weekday = linechart_data_weekday.groupby(['HOUR', 'Cluster'])['OCCUPANCY_PCT'].mean()
linechart_data_weekday  = linechart_data_weekday.reset_index()
y1 = linechart_data_weekday['OCCUPANCY_PCT'].values
x1 = linechart_data_weekday['HOUR'].values
labels1 = linechart_data_weekday['Cluster'].values
colours1 = linechart_data_weekday['Cluster'].values
df1 = pd.DataFrame(dict(x=x1, y=y1, label=labels1))
groups1 = df1.groupby('label')

#Saturday
linechart_data_saturday = linechart_data[linechart_data['DAY_TYPE'] == 'Saturday']
linechart_data_saturday = linechart_data_saturday.groupby(['HOUR', 'Cluster'])['OCCUPANCY_PCT'].mean()
linechart_data_saturday  = linechart_data_saturday.reset_index()
y2 = linechart_data_saturday['OCCUPANCY_PCT'].values
x2 = linechart_data_saturday['HOUR'].values
labels2 = linechart_data_saturday['Cluster'].values
colours2 = linechart_data_saturday['Cluster'].values
df2 = pd.DataFrame(dict(x=x2, y=y2, label=labels2))
groups2 = df2.groupby('label')

#Sunday
linechart_data_sunday = linechart_data[linechart_data['DAY_TYPE'] == 'Sunday']
linechart_data_sunday = linechart_data_sunday.groupby(['HOUR', 'Cluster'])['OCCUPANCY_PCT'].mean()
linechart_data_sunday  = linechart_data_sunday.reset_index()
y3 = linechart_data_sunday['OCCUPANCY_PCT'].values
x3 = linechart_data_sunday['HOUR'].values
labels3 = linechart_data_sunday['Cluster'].values
colours3 = linechart_data_sunday['Cluster'].values
df3 = pd.DataFrame(dict(x=x3, y=y3, label=labels3))
groups3 = df3.groupby('label')

import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

SMALL_SIZE = 10
MEDIUM_SIZE = 10
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for name, group in groups1:
    axs[0].plot(group.x, group.y, label=name)
    axs[0].set_title('Weekday')
    axs[0].set_xlabel('Hour')
    axs[0].set_ylabel('Occupancy %')
    #fig.suptitle('This is a somewhat long figure title', fontsize=16)

for name, group in groups2:
    axs[1].plot(group.x, group.y, label=name)
    axs[1].set_title('Saturday')
    axs[1].set_xlabel('Hour')
    axs[1].set_ylabel('Occupancy %')
    

for name, group in groups3:
    axs[2].plot(group.x, group.y, label=name)
    axs[2].set_title('Sunday')
    axs[2].set_xlabel('Hour')
    axs[2].set_ylabel('Occupancy %')
    axs[2].legend()

plt.show()

In [ ]:
# Impact of Stations
join_table= merged_with_clusters.groupby(['STATION ID','NAME', 'DATE']).agg(rain=('rain', 'sum'), TOTAL_CHANGES=('BIKE_ARR_DEP_ABS', 'sum'))
join_table =join_table.reset_index()
join_table['WET/DRY DAY'] = np.where(join_table['rain'] > 3, "Wet", "Dry")
join_table = join_table.drop(['rain'], axis = 1)
join_table =join_table.reset_index()
merged_with_clusters_wetdry = pd.merge(merged_with_clusters, join_table, on = ['STATION ID', 'NAME', 'DATE'], how = 'left')

wetday_df= merged_with_clusters_wetdry.groupby(['STATION ID', 'NAME', 'WET/DRY DAY']).agg(AVG_CHANGES=('TOTAL_CHANGES', 'mean'))
wetday_df =wetday_df.reset_index()
difference_df = wetday_df.pivot(index=['NAME'], columns='WET/DRY DAY', values='AVG_CHANGES').reset_index()
difference_df['Change'] = difference_df['Dry'] - difference_df['Wet']
difference_df.sort_values(by = 'Change', ascending=False).head(30)

In [ ]:
import pandas as pd
y = wetday_df['AVG_CHANGES'].values
x = wetday_df['STATION ID'].values
labels = wetday_df['WET/DRY DAY'].values
colours = wetday_df['WET/DRY DAY'].values
df = pd.DataFrame(dict(x=x, y=y, label=labels))
groups = df.groupby('label')

SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


fig, ax = plt.subplots(figsize=(30, 10))
#ax.set_color_cycle(colors)
ax.margins(0.05)
for name, group in groups:
    ax.plot(group.x, group.y,  marker='o',  linestyle='', ms=15, label=name)
ax.legend(numpoints=1, loc='upper left')
plt.xlabel("Station ID")
plt.ylabel("Avg Rentals/Day")
plt.show()

### ML Occupancy Percentage Classifier

In [ ]:
ml_classifier_data = merged_with_clusters[merged_with_clusters['REBALANCING'] < 1] #exclude days where rebalancing took place
ml_classifier_data = ml_classifier_data[['STATION ID', 'OCCUPANCY_PCT' , 'dry', 'warm', 'DAY_NUMBER', 'HOUR', 'MONTH']]


def bin_occupancy(x):
    if x < 0.1:
        return 0
    elif x < 0.1:
        return 0.1
    elif x < 0.2:
        return 0.1
    elif x < 0.3:
        return 0.1
    elif x < 0.4:
        return 0.1
    elif x < 0.5:
        return 0.1
    elif x < 0.6:
        return 0.1
    elif x < 0.7:
        return 0.1
    elif x < 0.8:
        return 0.1
    else:
        return 0.2

ml_classifier_data["OCC_GROUP"] = ml_classifier_data['OCCUPANCY_PCT'].apply(bin_occupancy)
ml_classifier_data["OCC_GROUP"] = ml_classifier_data["OCC_GROUP"] * 10
ml_classifier_data["OCC_GROUP"] = ml_classifier_data["OCC_GROUP"].astype(int)
ml_classifier_data.dropna(inplace = True)
msk = np.random.rand(len(ml_classifier_data)) < 0.8
train = ml_classifier_data[msk]
test = ml_classifier_data[~msk]
print(len(train))
print(len(test))

X_train = train.drop(['OCCUPANCY_PCT', "OCC_GROUP"], axis = 1)
X_test = test.drop(['OCCUPANCY_PCT', "OCC_GROUP"], axis = 1)
Y_train = train[["OCC_GROUP"]] 
Y_test = test[["OCC_GROUP"]]

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,Y_train)

Y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

#pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
print(sklearn.metrics.classification_report(Y_test,Y_pred))

In [ ]:
## show feature importances 

feature_imp = pd.Series(clf.feature_importances_,index=['STATION ID','DRY', 'WARM', 'DAY_NUMBER', 'HOUR', 'MONTH']).sort_values(ascending=False)
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()